In [1]:
!ls


sample_data


In [2]:
!git clone https://huggingface.co/phanikartcs/mahabharata_tatparya_nirnaya1


Cloning into 'mahabharata_tatparya_nirnaya1'...
remote: Enumerating objects: 46, done.
remote: Counting objects: 100% (42/42), done.
remote: Compressing objects: 100% (42/42), done.
remote: Total 46 (delta 5), reused 0 (delta 0), pack-reused 4 (from 1)
Unpacking objects: 100% (46/46), 1.74 MiB | 2.57 MiB/s, done.
Filtering content: 100% (20/20), 131.24 MiB | 73.20 MiB/s, done.


In [3]:
%cd mahabharata_tatparya_nirnaya1/mahabharata_tatparya_nirnaya_model1

/content/mahabharata_tatparya_nirnaya1/mahabharata_tatparya_nirnaya_model1


In [4]:
!ls

adapter_config.json	   checkpoint-2542   special_tokens_map.json
adapter_model.safetensors  checkpoint-3813   tokenizer_config.json
added_tokens.json	   merges.txt	     tokenizer.json
chat_template.jinja	   README.md	     vocab.json
checkpoint-1271		   requirements.txt


In [23]:
!pip install -q transformers accelerate peft bitsandbytes

In [24]:
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import PeftModel
import torch

In [40]:
base_model = "Qwen/Qwen1.5-1.8B"
lora_model = "/content/mahabharata_tatparya_nirnaya1/mahabharata_tatparya_nirnaya_model1"

tokenizer = AutoTokenizer.from_pretrained(base_model)
tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(
    base_model,
    device_map="auto",
    load_in_4bit=True
)

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


In [41]:
model = PeftModel.from_pretrained(model, lora_model)

In [76]:
def build_prompt(instruction):
    return f"""### Instruction:
{instruction}

### Response:
"""

prompt = build_prompt("Who killed Vali?")
inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

outputs = model.generate(
    **inputs,
    max_new_tokens=256,
    do_sample=True,
    temperature=0.1,
    eos_token_id=tokenizer.eos_token_id
)

print(tokenizer.decode(outputs[0], skip_special_tokens=True))

Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.


### Instruction:
Who killed Vali?

### Response:
Bhima
